# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [1]:
import numpy as np

In [2]:
def get_threshold(feature, y):
    ind = np.argsort(feature)    
    best_thr = np.min(feature)
    
    left_sum_sq, left_sum, left_n = 0, 0, 0
    right_sum_sq = np.sum(np.power(y, 2))
    right_sum = np.sum(y) 
    right_n = y.shape[0]
    
    left_d = 0
    right_d = right_sum_sq - (right_sum ** 2) / right_n
    f_score = left_d + right_d
    
    for i in ind:
        left_sum_sq += y[i] * y[i]
        left_sum += y[i]
        left_n += 1
        
        right_sum_sq -= y[i] * y[i]
        right_sum -= y[i]
        right_n = max(1, right_n - 1)
        
        left_d = left_sum_sq - (left_sum ** 2) / left_n
        right_d = right_sum_sq - (right_sum ** 2) / right_n
        cur_f_score = left_d + right_d
        if cur_f_score < f_score:
            f_score = cur_f_score
            best_thr = feature[i]
        
    return f_score, best_thr


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None # среднее значение слева и справа, для sklearn
    for j in range(X.shape[1]):
        f_score, f_thr = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_error = f_score
            best_f = j
            best_thr = f_thr
    left_count = np.sum(np.where(X[:, best_f] <= best_thr, 1, 0))
    left_mean = np.sum(np.where(X[:, best_f] <= best_thr, y, 0)) / max(1, left_count)
    
    right_count = np.sum(np.where(X[:, best_f] > best_thr, 1, 0))
    right_mean = np.sum(np.where(X[:, best_f] > best_thr, y, 0)) / max(1, right_count)
    return best_f, best_thr, left_mean, right_mean


def predict_stump(X, best_f, best_thr):
    left_count = np.sum(np.where(X[:, best_f] <= best_thr, 1, 0))
    left_mean = np.sum(np.where(X[:, best_f] <= best_thr, y, 0)) / max(1, left_count)
    
    right_count = np.sum(np.where(X[:, best_f] > best_thr, 1, 0))
    right_mean = np.sum(np.where(X[:, best_f] > best_thr, y, 0)) / max(1, right_count)
    
    return np.where(X[:, best_f] <= best_thr, left_mean, right_mean)

In [3]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [5]:
best_f, best_thr, left_mean, right_mean = train_stump(X, y)
y_pred = predict_stump(X, best_f, best_thr)

In [6]:
mean_squared_error(y_pred, y)

46.19909167710848